In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
from analysts_tools.redash_methods import *

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter

In [2]:
df_eagle = get_fresh_query_result("https://internal-redash.federate.frubana.com/",177832,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [3]:
df_offers = get_fresh_query_result("https://internal-redash.federate.frubana.com/",177833,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [4]:
df_bench = get_fresh_query_result("https://internal-redash.federate.frubana.com/",179153,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [5]:
query = """
SELECT DISTINCT 
  --qp.id,
  --qp.name,
  p.region_code,
  --p.name,
  p.name AS sku_name,
  bs2.name AS card_name,
  bs.addl_product_id AS card_id,
  p.product_id AS product_id,
  bs.sku_id,
  p.sku,
  bcat2.name as cat,
  bcat.name as subcat
FROM postgres_main_co."purchase_orders.products"                        p
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"              bs      ON bs.upc = p.sku
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_product"           bp      ON bp.product_id = bs.addl_product_id --Conectar sku con la tarjeta
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_sku"               bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"     bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL --Relaciones categorias 
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"          bcat    ON bp.default_category_id = bcat.category_id --Nombre subcategoria
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"          bcat2   ON bcx.category_id = bcat2.category_id --Nombre categoria

where
  p.deleted_at is null
  and p.parent_id is null
  and bs.archived = 'N'
  and bs.sndbx_id is null
  and bp.archived = 'N'
  and bs.catalog_disc < 0
  and ((bp.archived = 'N' and bp.sndbx_id is null) or (bp.archived is null))
  """
df_aux1 = read_connection_data_warehouse.runQuery(query)

In [6]:
query = """
SELECT  close_date
,       region_code
,       product_id
,CASE
    WHEN region_code IN ('SPO','BHZ','CWB','VCP') THEN SUM(stockout_correction * avg_price_without_discount)/4.75
    WHEN region_code IN ('BOG','BAQ','MDE') THEN SUM(stockout_correction * avg_price_without_discount)/3776
    WHEN region_code IN ('CMX') THEN SUM(stockout_correction * avg_price_without_discount)/19.65
    ELSE SUM(stockout_correction * avg_price_without_discount)
END AS gmv_lost
FROM    lnd_ops.dp_input_historical_product_information
WHERE   close_date >= CURRENT_DATE - 60
        AND warehouse <> 'ALL'
        --AND region_code = 'BOG'
GROUP BY 1, 2, 3
HAVING  SUM(stockout_correction * avg_price_without_discount) > 0
  """
df_so = read_connection_data_warehouse.run_read_prod_query(query)

In [7]:
query =  """
SELECT DISTINCT
    s.identifier_value AS region_code,
    DATE(fs.order_submitted_date) AS submit_date,
    cat.parent_description as category,
    cat.description AS subcategory,
    dp.card_id,
    dp.card_description AS product_name,
    AVG(fs.product_price + (fs.product_tax/fs.product_quantity_x_step_unit)) AS product_price,
    AVG(product_price_discount + (fs.product_tax/fs.product_quantity_x_step_unit)) AS product_price_discount--,
    --AVG(pb.price) AS benchmark
    
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id  
--INNER JOIN dpr_shared.dim_stock_unit         su  ON su.card_id = dp.card_id
--INNER JOIN dpr_pricing.snap_daily_benchmark_competitor pb ON pb.dim_stock_unit = su.stock_unit_id AND fs.dim_submitted_date = pb.dim_benchmark_day

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id  IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND fs.product_quantity_x_step_unit > 0
    AND DATE(fs.order_submitted_date) >= DATE_TRUNC('day', date(getdate()) - interval '60 day')
GROUP BY 1,2,3,4,5,6
"""
data_ventas_aux = run_read_dwd_query(query)
data_ventas_aux[["product_price", "product_price_discount"]] = data_ventas_aux[["product_price", "product_price_discount"]].astype(float)
data_ventas_aux['submit_date'] = pd.to_datetime(data_ventas_aux['submit_date']) 

In [8]:
df_so = pd.merge(df_so, df_aux1[['region_code','product_id','card_id']],  how='left', left_on=['region_code','product_id'], right_on = ['region_code','product_id'])

In [9]:
df = pd.merge(df_eagle, df_aux1[['region_code','product_id','card_id','sku_name','cat','subcat']],  how='left', left_on=['region_code','product_id'], right_on = ['region_code','product_id'])

In [10]:
df = df.dropna()

In [11]:
# Assuming df is your DataFrame
def calculate_fit_for_discount(row):
    if row['cat'] in ['Frutas e Verduras', 'Frutas & Verduras']:
        if row['clearence_days'] < 1:
            return 'FALSE'
        elif 1 <= row['clearence_days'] < 2:
            return 'check_with_supply'
        else:
            return 'TRUE'
    elif row['cat'] in ['Abarrotes', 'Abarrotes & Despensa', 'Mercearia', 'Bebidas', 'Congelados', 'Despensa']:
        if row['clearence_days'] <= 3:
            return 'FALSE'
        elif 3 < row['clearence_days'] <= 5:
            return 'check_with_supply'
        else:
            return 'TRUE'
    elif row['cat'] in ['Lácteos & Huevos', 'Lácteos Y Huevos', 'Laticínios e Ovos', 'Carne, Pollo & Pescados', 'Carnes, Aves e Peixes']:
        if row['clearence_days'] <= 1:
            return 'FALSE'
        elif 1 < row['clearence_days'] <= 3:
            return 'check_with_supply'
        else:
            return 'TRUE'
    elif row['cat'] in ['Aseo e Higiene', 'Desechables', 'Descartáveis', 'Limpeza e Higiene']:
        if row['clearence_days'] <= 2:
            return 'FALSE'
        elif 2 < row['clearence_days'] <= 4:
            return 'check_with_supply'
        else:
            return 'TRUE'
    else:
        return 'FALSE'

df['fit_for_discount'] = df.apply(calculate_fit_for_discount, axis=1)

In [12]:
# Convert date columns to a common format
df_so['close_date'] = pd.to_datetime(df_so['close_date'])
df['operation_date'] = pd.to_datetime(df['operation_date'])
df_bench['fecha'] = pd.to_datetime(df_bench['fecha'])
df_offers['created_at'] = pd.to_datetime(df_offers['created_at'])

In [13]:
df3 = pd.merge(df_offers, df[['region_code','operation_date','card_id','clearence_days','fit_for_discount']],  how='left', left_on=['city','created_at','card_id'], right_on = ['region_code','operation_date','card_id'])

In [14]:
df3 = df3.dropna().reset_index(drop=True)

In [15]:
import pandas as pd
from datetime import timedelta

# Assuming df3 is your offers DataFrame and df_so is your stock DataFrame

# Convert date columns to datetime format
df3['created_at'] = pd.to_datetime(df3['created_at'])
df_so['close_date'] = pd.to_datetime(df_so['close_date'])

# Perform the left join
merged_df = df3.merge(df_so[['card_id', 'close_date', 'gmv_lost']], how='left', on='card_id')

# Add condition to create null values for rows that don't match the condition
merged_df.loc[
    ~(
        (merged_df['close_date'] >= merged_df['created_at']) &
        (merged_df['close_date'] <= merged_df['created_at'] + timedelta(days=5))
    ),
    ['close_date', 'gmv_lost']
] = [None, None]


merged_df['close_date'] = merged_df['close_date'].fillna(merged_df['created_at'])
merged_df['gmv_lost'] = merged_df['gmv_lost'].fillna(0)
#Filter rows based on conditions
merged_df = merged_df[
    ((merged_df['close_date'] >= merged_df['created_at']) &
     (merged_df['close_date'] <= merged_df['created_at'] + timedelta(days=5)))
]

In [16]:
merged_df = merged_df.drop_duplicates()

In [17]:
# Custom function to count non-zero and non-null values
def custom_count(series):
    return len([x for x in series if pd.notna(x) and x != 0])

# Assuming 'merged_df' is your DataFrame
result_df = merged_df.groupby(by=['city', 'responsable', 'created_at',
       'card_id', 'name', 'cat', 'subcat', 'region_code', 'operation_date',
       'clearence_days', 'fit_for_discount']).agg({'gmv_lost': [np.sum, custom_count]}).reset_index()

# Rename columns for clarity
result_df.columns = ['city', 'responsable', 'created_at',
       'card_id', 'name', 'cat', 'subcat', 'region_code', 'operation_date',
       'clearence_days', 'fit_for_discount', 'total_gmv_lost', 'count_non_zero']

In [18]:
data_ventas_aux["dct"] = 1-(data_ventas_aux.product_price_discount/data_ventas_aux.product_price)

In [19]:
df2 = data_ventas_aux.copy()

In [20]:
# Assuming your DataFrame is named 'df2'
# Sort the DataFrame by 'card_id' and 'submit_date' in ascending order
df2.sort_values(by=['card_id', 'submit_date'], inplace=True)

# Calculate the percentage change for 'gpi', 'dct', and 'npi'
df2['gpi_percent_change'] = df2.groupby('card_id')['product_price'].pct_change() #* 100
df2['npi_percent_change'] = df2.groupby('card_id')['product_price_discount'].pct_change() #* 100
df2['dct_percent_change'] = df2.groupby('card_id')['dct'].pct_change() #* 100

# Drop the first row for each 'card_id' since percentage change is not available
df2 = df2.dropna()

# Reset the index if needed
df2.reset_index(drop=True, inplace=True)

In [21]:
# Calculate the rolling average for each card_id and submit_date
rolling_period = 3  # Adjust this value as needed

for col in ['product_price', 'product_price_discount', 'dct']:
    df2[f'{col}_avg_t-3'] = df2.groupby('card_id')[col].transform(lambda x: x.rolling(rolling_period).mean())
    df2[f'{col}_avg_t+3'] = df2.groupby('card_id')[col].transform(lambda x: x.shift(-2).rolling(rolling_period).mean())
    df2[f'{col}_variation'] = (df2[f'{col}_avg_t+3'] - df2[f'{col}_avg_t-3']) / df2[f'{col}_avg_t-3']

    
df2 = df2.dropna()

In [22]:
df2.to_excel("C2.xlsx")

In [23]:
df_final = pd.merge(result_df, df2[['region_code','submit_date','card_id','product_price_avg_t-3',
       'product_price_avg_t+3', 'product_price_variation', 'product_price_discount_avg_t-3',
       'product_price_discount_avg_t+3', 'product_price_discount_variation','dct_avg_t-3',
       'dct_avg_t+3', 'dct_variation']],  how='left', left_on=['city','created_at','card_id'], right_on = ['region_code','submit_date','card_id'])

In [24]:
df_final = df_final.dropna()

In [25]:
df_final

,city,responsable,created_at,card_id,name,cat,subcat,region_code_x,operation_date,clearence_days,...,submit_date,product_price_avg_t-3,product_price_avg_t+3,product_price_variation,product_price_discount_avg_t-3,product_price_discount_avg_t+3,product_price_discount_variation,dct_avg_t-3,dct_avg_t+3,dct_variation
290,BAQ,ADDITIONAL INVESTMENT,2023-07-31,-201313,Papa Blanca Segunda,Frutas & Verduras,Tubérculos,BAQ,2023-07-31,318.0,...,2023-07-31,950.000000,950.000000,0.000000e+00,945.402436,932.583333,-1.355941e-02,0.004840,0.018333,2.788238e+00
293,BAQ,ADDITIONAL INVESTMENT,2023-07-31,-200953,Huevos Rojos A x 30 und,Lácteos & Huevos,Huevos,BAQ,2023-07-31,1.0,...,2023-07-31,16485.033571,15931.214683,-3.359525e-02,15428.996841,15265.436861,-1.060082e-02,0.064068,0.041601,-3.506731e-01
294,BAQ,ADDITIONAL INVESTMENT,2023-07-31,-200951,Aceite Riquísimo 1Lt,Abarrotes,Aceites,BAQ,2023-07-31,1.0,...,2023-07-31,10085.823333,10085.823213,-1.193461e-08,9224.523333,9224.523213,-1.304895e-08,0.086096,0.086096,1.804834e-08
298,BAQ,ADDITIONAL INVESTMENT,2023-07-31,-200859,Pasta Spaguetti La Muñeca x 250gr,Despensa,Pasta,BAQ,2023-07-31,0.0,...,2023-07-31,1909.120000,1911.004954,9.873417e-04,1787.520000,1827.100954,2.214294e-02,0.063796,0.043987,-3.105008e-01
304,BAQ,ADDITIONAL INVESTMENT,2023-07-31,-200683,Ajo,Frutas & Verduras,Verduras,BAQ,2023-07-31,-1.0,...,2023-07-31,89888.888889,90000.000000,1.236094e-03,89773.888889,89857.916667,9.359935e-04,0.001278,0.001579,2.355072e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42636,VCP,SUPER HOOKS,2023-09-19,413514,Filé de Peito de Frango - Bello Alimentos,"Carnes, Aves e Peixes",Frango,VCP,2023-09-19,1294.0,...,2023-09-19,13.124167,13.153958,2.269985e-03,12.338611,12.366736,2.279430e-03,0.059856,0.059847,-1.481943e-04
42638,VCP,SUPER HOOKS,2023-09-20,328555,Laranja Pera,Frutas e Verduras,Frutas,VCP,2023-09-20,0.0,...,2023-09-20,2.287549,2.310000,9.814426e-03,2.159399,2.295357,6.296068e-02,0.056153,0.006339,-8.871111e-01
42640,VCP,SUPER HOOKS,2023-09-22,326496,Molho de Tomate Tarantella Sachê 2kg,Mercearia,Massas e Molhos,VCP,2023-09-22,18.0,...,2023-09-22,10.631786,10.631508,-2.612710e-05,9.995278,9.995000,-2.779090e-05,0.059868,0.059870,2.036040e-05
42642,VCP,SUPER HOOKS,2023-09-22,413552,Café Extraforte Almofada Moraes 500g,Mercearia,"Café, Chocolates e Infusões",VCP,2023-09-22,2.0,...,2023-09-22,13.216222,13.067333,-1.126562e-02,12.158444,12.020667,-1.133186e-02,0.080037,0.080100,7.953205e-04


In [26]:
df_final.to_excel("B2.xlsx")

In [27]:
#result_df.to_excel("A2.xlsx")